<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Planet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Planet
[Github Repository](https://github.com/progirep/planet)

In [1]:
%%capture

# Clone the repo
!git clone https://github.com/progirep/planet.git

In [2]:
%%capture
# we need a script to convert the neural network description in to the "prototxt" format (produced by Caffe) into a json file
!cd tools
!wget https://gist.github.com/progirep/fd7d2dc120862faa984a70f503611013/raw/260e1e76cebd0ea58bf1a03b64c3f1e0002fc677/csv_to_hdf5_supervised_classification.py 

# we need a second script to generate a database in "HDF5" format from comma-separated value files
!wget https://raw.githubusercontent.com/vadimkantorov/caffemodel2json/3a8fd443bf1596dad5f517aecdef08a81bf73bfe/caffemodel2json.py

In [3]:
%%capture

# install packages in order to build PLANET
!sudo apt-get install libglpk-dev
!sudo apt-get install qt5-qmake
!sudo apt-get install valgrind
!sudo apt-get install libltdl-dev
!sudo apt-get install protobuf-compiler

In [4]:
%%capture

# compile the source code
%cd planet/src
%ls
!qmake Tool.pro
!make

In [5]:
%%capture

# install Caffe and Yices
# change root path of #CAFFE and #YICES
!sudo apt install caffe-cuda
!sudo add-apt-repository ppa:sri-csl/formal-methods -qq
!sudo apt-get update
!sudo apt-get install yices2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  caffe-tools-cuda cython3 fonts-lyx javascript-common libblosc1
  libcaffe-cuda1 libcublas9.1 libcudart9.1 libcurand9.1 libgflags2.2
  libgoogle-glog0v5 libjs-jquery libjs-jquery-ui libleveldb1v5 liblmdb0
  python-matplotlib-data python-tables-data python3-bs4 python3-caffe-cuda
  python3-chardet python3-cycler python3-dateutil python3-decorator
  python3-gflags python3-h5py python3-html5lib python3-ipython
  python3-ipython-genutils python3-leveldb python3-lxml python3-matplotlib
  python3-networkx python3-nose python3-numexpr python3-numpy python3-olefile
  python3-pandas python3-pandas-lib python3-pexpect python3-pickleshare
  python3-pil python3-pkg-resources python3-prompt-toolkit python3-protobu

In [6]:
# envs probably useless
%env YICES=/usr
%env CAFFE=/usr
%cd /content/planet/casestudies/MNIST/

env: YICES=/usr
env: CAFFE=/usr
/content/planet/casestudies/MNIST


In [8]:
%%bash

export YICES=/usr
export CAFFE=/sr
set -e
caffe train --solver=lenet_solver.prototxt $@ 

I0601 10:22:35.211846  2800 caffe.cpp:211] Use CPU.
I0601 10:22:35.619149  2800 solver.cpp:44] Initializing solver from parameters: 
test_iter: 100
test_interval: 1000
base_lr: 0.01
display: 1000
max_iter: 200000
lr_policy: "inv"
gamma: 0.0001
power: 0.75
momentum: 0.9
weight_decay: 0.0005
snapshot: 100000
snapshot_prefix: "snapshots/"
solver_mode: CPU
net: "lenet_train_test.prototxt"
train_state {
  level: 0
  stage: ""
}
I0601 10:22:35.619308  2800 solver.cpp:87] Creating training net from net file: lenet_train_test.prototxt
I0601 10:22:35.619482  2800 net.cpp:294] The NetState phase (0) differed from the phase (1) specified by a rule in layer mnist
I0601 10:22:35.619563  2800 net.cpp:51] Initializing net from parameters: 
name: "LeNet"
state {
  phase: TRAIN
  level: 0
  stage: ""
}
layer {
  name: "mnist"
  type: "Data"
  top: "data"
  top: "label"
  include {
    phase: TRAIN
  }
  transform_param {
    scale: 0.00390625
  }
  data_param {
    source: "mnist_train_lmdb"
    batch_

Current Problem:
https://github.com/BVLC/caffe/issues/2780
> Check failed: mdb_status == 0



#Current Problems
* csv_to_hdf5_supervised_classificatoin.py doesn't work, because it's Python2
* Error with caffe train - not sure why -https://github.com/BVLC/caffe/issues/2780

#TODO's
* Try to do a local jupyter notebook instead of colab
* Continue the "Running the Collision Avoidance Benchmarks" Section

